Documentación Técnica del Despliegue: Sistema CAU Ingeniería
1. Arquitectura del Sistema
El proyecto se ha desplegado utilizando una arquitectura de microservicios contenerizados alojados en una instancia EC2 de AWS, conectados a una base de datos relacional gestionada externamente en AWS RDS.

Servidor de Aplicación: AWS EC2 (Ubuntu Linux).

Orquestación: Docker & Docker Compose.

Base de Datos: AWS RDS (PostgreSQL).

Cliente de Gestión BD: pgAdmin 4 (Escritorio).

2. Infraestructura de Datos (Capa de Persistencia)
Se optó por separar la base de datos del ciclo de vida de los contenedores para garantizar la persistencia y seguridad de los datos.

Servicio: Amazon Relational Database Service (RDS).

Motor: PostgreSQL.

Identificador de Instancia: bbdd-cau.

Endpoint de Conexión: bbdd-cau.cfeysagwgftt.eu-north-1.rds.amazonaws.com.

Puerto: 5432.

Configuración de Red:

Se habilitó el acceso público para permitir la administración remota y la conexión desde el servidor de aplicaciones.

Security Groups (Cortafuegos): Se configuró una regla de entrada (Inbound Rule) para el tráfico TCP en el puerto 5432 desde 0.0.0.0/0, permitiendo la conexión tanto desde el entorno de desarrollo (pgAdmin local) como desde el despliegue en producción (Docker en EC2).

3. Despliegue de la Aplicación (Capa Lógica y Presentación)
La aplicación se ejecuta dentro de contenedores Docker para asegurar la consistencia entre entornos.

3.1. Estructura de Contenedores (docker-compose.yaml)
El despliegue consta de dos servicios principales definidos en el archivo de composición:

Frontend (cau_frontend_container):

Servidor web Nginx configurado como proxy inverso y servidor de archivos estáticos.

Expuesto en el puerto 80.

Backend (cau_backend_container):

API REST desarrollada en Python/Flask.

Expuesto internamente en el puerto 5000.

Gestión de Configuración: Utiliza variables de entorno inyectadas mediante un archivo .env para manejar credenciales y endpoints sensibles.

3.2. Integración Backend - Base de Datos
Para conectar el contenedor de backend (situado en EC2) con la base de datos (RDS), se configuró el archivo .env del backend para apuntar al host remoto en lugar de a un contenedor local.

Variables de entorno configuradas:

Bash

DB_USER=postgres
DB_PASSWORD=[CREDENCIAL_SEGURA]
DB_HOST=bbdd-cau.cfeysagwgftt.eu-north-1.rds.amazonaws.com
DB_PORT=5432
DB_NAME=postgres
Esta configuración permite que el ORM (SQLAlchemy) del backend establezca sesiones de lectura/escritura directamente contra la instancia de AWS RDS.

4. Procedimiento de Puesta en Marcha
Se ejecutaron los siguientes pasos técnicos en la terminal del servidor Ubuntu para finalizar el despliegue:

Inyección de Credenciales: Se generó el archivo de variables de entorno con la configuración de producción:

Bash

printf "DB_USER=postgres\nDB_PASSWORD=...\nDB_HOST=...\n..." > backend/.env
Construcción y Arranque: Se forzó la recreación de los contenedores para aplicar los cambios de configuración de red:

Bash

docker compose up -d backend frontend
Verificación de Conectividad: Se validó la comunicación mediante logs del contenedor, confirmando que el backend recibía respuestas 200 OK de la base de datos tras la apertura de puertos en el Security Group de AWS.